# VisionTextDualEncoder and CLIP-base model training - Pytorch
This notebook shows how to fine-tune a pretrained HuggingFace CLIP-base PyTorch model with AWS Trainium (trn1 instances) using NeuronSDK.
The original implementation is provided by HuggingFace.

The example has 2 stages:
1. First compile the model using the utility `neuron_parallel_compile` to compile the model to run on the AWS Trainium device.
1. Run the fine-tuning script to train the model based on image classification task. The training job will use 32 workers with data parallel to speed up the training.

It has been tested and run on trn1.32xlarge instance

**Reference:** 

https://huggingface.co/openai/clip-vit-base-patch32

## 1) Install dependencies

Verify that this Jupyter notebook is running the Python kernel environment that was set up according to the [PyTorch Installation Guide](https://awsdocs-neuron.readthedocs-hosted.com/en/latest/general/setup/torch-neuronx.html#setup-torch-neuronx). You can select the kernel from the 'Kernel -> Change Kernel' option on the top of this Jupyter notebook page.

In [ ]:
#Supresses tokenizer warnings making errors easier to detect
%env TOKENIZERS_PARALLELISM=True
#Install Neuron Compiler and Neuron/XLA packages
%pip install -U "numpy<=1.20.0" "protobuf<4" "transformers==4.27.3" datasets scikit-learn 
# use --force-reinstall if you're facing some issues while loading the modules
# now restart the kernel again

## 2) Set the parameters

In [ ]:
# Common Parameters
text_model_name = "roberta-base"
env_var_options = "MALLOC_ARENA_MAX=64 XLA_USE_BF16=1 NEURON_CC_FLAGS=\"--cache_dir=./compiler_cache\""
num_workers = 32
task_name = "contrastive-image-text"
dataset_name = "ydshieh/coco_dataset_script"
transformers_version = "4.27.3"
learning_rate = 5e-5
dataloader_num_workers = 2
device_prefetch_size = 2
host_to_device_transfer_threads = 4

In [ ]:
# Parameters for HuggingFace CLIP-base model
model_name = "openai/clip-vit-base-patch32"
model_base_name = "clip-base"
per_device_train_batch_size = 64
per_device_eval_batch_size = 64

## 3) Download COCO dataset
This example uses COCO dataset (2017) through a custom dataset script, which requires users to manually download the COCO dataset before training.

In [ ]:
!mkdir -p data
!wget http://images.cocodataset.org/zips/train2017.zip -P data
!wget http://images.cocodataset.org/zips/val2017.zip -P data
!wget http://images.cocodataset.org/zips/test2017.zip -P data
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip -P data
!wget http://images.cocodataset.org/annotations/image_info_test2017.zip -P data

## 4) Compile the model with neuron_parallel_compile

In [ ]:
%%time
import subprocess
print("Compile model")
COMPILE_CMD = f"""{env_var_options} neuron_parallel_compile torchrun --nproc_per_node={num_workers} \
    run_clip.py \
    --model_name_or_path {model_name} \
    --text_model_name_or_path {text_model_name} \
    --data_dir $PWD/data \
    --dataset_config_name=2017 \
    --dataset_name {dataset_name} \
    --image_column image_path \
    --caption_column caption \
    --do_train \
    --max_steps 10 \
    --num_train_epochs 2 \
    --per_device_train_batch_size {per_device_train_batch_size} \
    --per_device_eval_batch_size {per_device_eval_batch_size} \
    --learning_rate {learning_rate} \
    --warmup_steps 0 \
    --weight_decay 0.1 \
    --save_strategy epoch \
    --save_total_limit 1 \
    --seed 1337 \
    --remove_unused_columns False \
    --overwrite_output_dir \
    --output_dir {model_base_name}-{task_name} \
    --device_prefetch_size {device_prefetch_size} \
    --host_to_device_transfer_threads {host_to_device_transfer_threads}"""

print(f'Running command: \n{COMPILE_CMD}')
if subprocess.check_call(COMPILE_CMD,shell=True):
   print("There was an error with the compilation command")
else:
   print("Compilation Success!!!")

## 5) Fine-tune the model

In [ ]:
%%time
print("Train model")
RUN_CMD = f"""{env_var_options} torchrun --nproc_per_node={num_workers} \
    run_clip.py \
    --model_name_or_path {model_name} \
    --text_model_name_or_path roberta-base \
    --data_dir $PWD/data \
    --dataset_config_name=2017 \
    --dataset_name {dataset_name} \
    --image_column image_path \
    --caption_column caption \
    --do_train \
    --do_eval \
    --num_train_epochs 2 \
    --per_device_train_batch_size {per_device_train_batch_size} \
    --per_device_eval_batch_size {per_device_eval_batch_size} \
    --learning_rate {learning_rate} \
    --warmup_steps 0 \
    --weight_decay 0.1 \
    --save_strategy epoch \
    --save_total_limit 1 \
    --seed 1337 \
    --remove_unused_columns False \
    --overwrite_output_dir \
    --output_dir {model_base_name}-{task_name} \
    --device_prefetch_size {device_prefetch_size} \
    --host_to_device_transfer_threads {host_to_device_transfer_threads}"""

print(f'Running command: \n{RUN_CMD}')
if subprocess.check_call(RUN_CMD,shell=True):
   print("There was an error with the fine-tune command")
else:
   print("Fine-tune Successful!!!")